
## SQL-Lösungen zu Aufgben Komplexe Anfragen

### Aufgabe 1
```sql
SELECT
    organization.name,
    SUM(country.population) AS total_population
FROM
    ismember
JOIN country ON ismember.country = country.code
JOIN organization ON ismember.organization = organization.abbreviation
GROUP BY organization.name
ORDER BY total_population DESC;
```

### Aufgabe 2
```sql
SELECT
    country.name AS country,
    geo_mountain.name AS mountain,
    geo_mountain.elevation
FROM
    country
JOIN geo_mountain ON geo_mountain.country = country.code
WHERE
    country.continent = 'America' AND geo_mountain.elevation IS NOT NULL
AND (geo_mountain.name, geo_mountain.elevation) IN (
    SELECT name, MAX(elevation)
    FROM geo_mountain
    GROUP BY country
)
ORDER BY geo_mountain.elevation DESC;
```

### Aufgabe 3
```sql
SELECT
    c1.name AS country,
    SUM(c2.population) AS neighbor_population
FROM
    country c1
JOIN borders b ON c1.code = b.country1
JOIN country c2 ON b.country2 = c2.code
GROUP BY c1.name;
```

### Aufgabe 4
```sql
SELECT
    AVG(latitude) AS avg_latitude,
    AVG(longitude) AS avg_longitude
FROM city;
```

### Aufgabe 5
```sql
SELECT
    country.name AS country,
    province.name AS province,
    province.population AS province_population,
    country.population AS country_population
FROM
    country
JOIN province ON province.country = country.code
WHERE
    ABS(province.population - country.population) > 0.1 * country.population;
```

### Aufgabe 6
```sql
SELECT
    religion.religion,
    ROUND(SUM(religion.percent * country.population / 100) * 100.0 / 
        (SELECT SUM(population) FROM country), 2) AS world_percent
FROM
    religion
JOIN country ON religion.country = country.code
GROUP BY religion.religion
ORDER BY world_percent DESC;
```

### Aufgabe 7 – Teil 1
```sql
WITH region AS (
    SELECT code FROM country
    WHERE code IN ('DZ', 'LAR')
    UNION
    SELECT country2 FROM borders WHERE country1 IN ('DZ', 'LAR')
    UNION
    SELECT country1 FROM borders WHERE country2 IN ('DZ', 'LAR')
)
SELECT
    SUM(country.population) / SUM(country.area) AS density
FROM country
WHERE code IN (SELECT code FROM region);
```

### Aufgabe 7 – Teil 2 (ohne Wüsten)
```sql
WITH region AS (
    SELECT code FROM country
    WHERE code IN ('DZ', 'LAR')
    UNION
    SELECT country2 FROM borders WHERE country1 IN ('DZ', 'LAR')
    UNION
    SELECT country1 FROM borders WHERE country2 IN ('DZ', 'LAR')
)
SELECT
    SUM(country.population) / SUM(country.area - COALESCE(desert.area, 0)) AS adjusted_density
FROM country
LEFT JOIN desert ON country.code = desert.country
WHERE country.code IN (SELECT code FROM region);
```

### Aufgabe 8
```sql
SELECT c1.name AS country1, c2.name AS country2
FROM country c1
JOIN country c2 ON c1.code < c2.code
WHERE c1.continent = 'Europe' AND c2.continent = 'Europe'
AND NOT EXISTS (
    SELECT * FROM (
        SELECT sea FROM country_sea WHERE country = c1.code
        EXCEPT
        SELECT sea FROM country_sea WHERE country = c2.code
    ) AS diff1
)
AND NOT EXISTS (
    SELECT * FROM (
        SELECT sea FROM country_sea WHERE country = c2.code
        EXCEPT
        SELECT sea FROM country_sea WHERE country = c1.code
    ) AS diff2
);
```

### Aufgabe 9
```sql
SELECT o.name
FROM organization o
WHERE NOT EXISTS (
    SELECT DISTINCT continent FROM country
    EXCEPT
    SELECT DISTINCT c.continent
    FROM ismember m
    JOIN country c ON m.country = c.code
    WHERE m.organization = o.abbreviation
);
```

### Aufgabe 10
```sql
SELECT o.name, SUM(b.length) AS total_border_length
FROM organization o
JOIN ismember m ON o.abbreviation = m.organization
JOIN borders b ON m.country = b.country1
WHERE b.country2 NOT IN (
    SELECT country FROM ismember m2 WHERE m2.organization = o.abbreviation
)
GROUP BY o.name;
```

### Aufgabe 11 – Teil 1
```sql
SELECT c.continent, o.name
FROM continent c
JOIN country co ON co.continent = c.continent
JOIN organization o
WHERE NOT EXISTS (
    SELECT * FROM country c1
    WHERE c1.continent = c.continent
    EXCEPT
    SELECT country FROM ismember WHERE organization = o.abbreviation
)
GROUP BY c.continent, o.name;
```

### Aufgabe 11 – Teil 2 (nur Länder mit Organisation)
```sql
SELECT c.continent, o.name
FROM continent c
JOIN country co ON co.continent = c.continent
JOIN organization o
WHERE NOT EXISTS (
    SELECT * FROM country c1
    WHERE c1.continent = c.continent AND c1.code IN (
        SELECT country FROM ismember
    )
    EXCEPT
    SELECT country FROM ismember WHERE organization = o.abbreviation
)
GROUP BY c.continent, o.name;
```

### Aufgabe 12
```sql
SELECT city.name, city.population, city.country
FROM city
JOIN (
    SELECT country, AVG(population) AS avg_pop
    FROM city
    GROUP BY country
) avg_cities ON city.country = avg_cities.country
WHERE city.population > avg_cities.avg_pop;
```

### Aufgabe 13
```sql
WITH highest_per_continent AS (
    SELECT continent, MAX(m.elevation) AS max_elevation
    FROM geo_mountain m
    JOIN country c ON m.country = c.code
    GROUP BY continent
)
SELECT m.name, m.elevation, c.continent
FROM geo_mountain m
JOIN country c ON m.country = c.code
JOIN highest_per_continent h ON c.continent = h.continent AND m.elevation = h.max_elevation
ORDER BY m.elevation ASC
LIMIT 1;
```
